# Regression Models for Fa Prediction using Descriptors Calculated with Mordred

## Materials and Method

- Libraries: NumPy, pandas, scikit-learn, matplotlib, RDKit, mordred and SHAP
- Dataset: Fraction of absorption (Fa) and Parmeability measured by Caco-2 cells (Papp), which were collected previous strudy (Esaki, et al., Journal of Phermeceutical Sciences, 2019)
- Descriptor calcularion: Mordred

### Library import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
print('numpy version: ', np.__version__)
print('pandas version: ', pd.__version__)
print('matplotlib version: ', matplotlib.__version__)

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import VarianceThreshold,  SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV, ElasticNet
print('scikit-learn version: ', sklearn.__version__)

from rdkit import Chem, rdBase
print('rdkit version: ', rdBase.rdkitVersion)

import shap
print('shap version: ', shap.__version__)

In [ ]:
target = 'Papp'

### Dataset Import

The dataset contained information on the chemical structure of 5567 compounds as SMILES strings. In this dataset, the number of Fa experimental values was 946, respectively. Owing to its accuracy, we used CORINA (ver. 4.4.0) to generate 3D structures of the chemical compounds as structure data format (SDF) .

We used the molecular descriptor calculator, Mordred, to calculate descriptors (1826 for 1D, 2D, and 3D). These descriptors were calculated for Papp dataset.

In [ ]:
df_3Ddescriptors = pd.read_csv('CBIJ_Esaki_et_al_Descriptor_Mordred_1D2D3D.csv', index_col=0)
df_3Ddescriptors.head(5)

In [ ]:
df_3Ddescriptors = df_3Ddescriptors.drop("Fa", axis=1)
df_3Ddescriptors.head(5)

## Definition of Functions

### Data preparation and Distribution conformation

The Papp dataset was randomly split into training (70%, 3087 compounds) and test set (30%, 1324 compounds) using the train_test_split function in scikit-learn. The Papp measurements ranged between 0.00016 and 880. We transformed these values to log10(Papp) to scatter the response variable.

In [ ]:
def data_separation(df, target):
    df = df.dropna(subset=[target])
    X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 2:], df[target],
                                                        train_size=0.7, random_state=0)
    return X_train, X_test, y_train, y_test


def distribution_conformation(df, target):
    X_train, X_test, y_train, y_test = data_separation(df, target)
    
    # remove descriptors with nan
    X_train = X_train.dropna(axis=1)
    X_test = X_test[X_train.columns]
    print(X_train.shape, X_test.shape, X_test.dropna(axis=1).shape)
    
    fig = plt.figure(figsize=(15, 3))
    
    for f in range(4):
        ax = fig.add_subplot(1, 4, f+1)
        if f == 0:
            ax.hist(y_train, bins=25, label='Training set')
            ax.hist(y_test, bins=25, label='Test set')
            ax.set_title('{}'.format(target))
        elif f == 1:
            if target == 'Fa':
                y_train[y_train == 0] = 0.01
                y_train[y_train == 1] = 0.99
                y_train = np.log(y_train/(1-y_train))
                y_test[y_test == 0] = 0.01
                y_test[y_test == 1] = 0.99
                y_test = np.log(y_test/(1-y_test))
            elif target == 'Papp':
                y_train = np.log(y_train)
                y_test = np.log(y_test)
            ax.hist(y_train, bins=25, label='Training set')
            ax.hist(y_test, bins=25, label='Test set')
            ax.set_title('Converted {}'.format(target))
        elif f == 2:
            ax.hist(X_train['MW'], range=(0,700), bins=25, label='Training set')
            ax.hist(X_test['MW'], range=(0,700), bins=25, label='Test set')
            ax.set_title('MW: {} data'.format(target))
        elif f == 3:
            ax.hist(X_train['SLogP'], range=(-10,10), bins=25, label='Training set')
            ax.hist(X_test['SLogP'], range=(-10,10), bins=25, label='Test set')
            ax.set_title('SLogP: {} data'.format(target))
        plt.legend(fontsize=12)
    plt.show()
    
    return X_train, X_test, y_train, y_test

### Descriptor preparation

We performed data preparation steps for descriptors of compounds in the training set. First, descriptors with nan were removed. Next, descriptors with small variance were removed using the VarianceThreshold function (threshold=1.0), and the retained descriptors were normalized using the StandardScaler function. Finally, effective descriptors for prediction were selected using ElasticNet. ElasticNetCV (cv=10, max_iter=1000000) selected proper alpha (penalty) and l1_ratio (norm ratio), using which we performed ElasticNet to retain proper descriptors.

In [ ]:
def descriptor_preparation(X_train, X_test, y_train):
    # Remove no variance descriptors
    var = VarianceThreshold(threshold=1.0).fit(X_train)
    X_train = X_train.loc[:, var.get_support()]
    X_test = X_test.loc[:, var.get_support()]
    
    # Standard Scaler
    ss = StandardScaler().fit(X_train)
    X_train_scd = pd.DataFrame(ss.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test_scd = pd.DataFrame(ss.transform(X_test), index=X_test.index, columns=X_test.columns)
    
    # Fill in NaNs with avereges
    train_averages = X_train_scd.mean()
    X_test_scd = X_test_scd.fillna(train_averages)
    
    # Descriptor Selection
    elastic = ElasticNetCV(cv=10, max_iter=1000000, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], n_jobs=-1).fit(X_train_scd, y_train)
    print(f'alpha: {elastic.alpha_}, l1_ratio: {elastic.l1_ratio_}')
    
    elastic_model = ElasticNet(alpha=elastic.alpha_, l1_ratio=elastic.l1_ratio_, max_iter=1000000).fit(X_train_scd, y_train)
    selected_descriptors = pd.DataFrame(elastic_model.coef_)
    selected_descriptors.index = X_train_scd.columns
    selected_descriptors = selected_descriptors[selected_descriptors[0] != 0]
    X_train_elastic = X_train_scd.loc[:, selected_descriptors.index]
    X_test_elastic = X_test_scd.loc[:, selected_descriptors.index]
    print(f'number of selected descriptor: {X_train_elastic.shape[1]}')
    
    return X_train_elastic, X_test_elastic

### Model construction

To construct Support Vector Regression (SVR) model for Papp prediction. SVR is a kernel function model where we employed the frequently used radial basis function kernel. Here, three parameters need to be optimized, which are C: regularization parameter, epsilon: insensitive zone, and gamma: kernel coefficient.

In [ ]:
from sklearn.svm import SVR

def model_construction_svr(X_train_elastic, y_train):    
    # SVR
    search_params = [{'C'       : [2**n for n in range(0, 7, 1)],
                      'epsilon' : [2**n for n in range(-5, 5, 1)],
                      'gamma'   : [2**n for n in range(-10, 0, 1)]}]
    gs_svr = GridSearchCV(SVR(kernel='rbf'),
                          search_params,
                          cv=10,
                          n_jobs=-1,
                          scoring='neg_mean_squared_error')
    gs_svr.fit(X_train_elastic, y_train)
    
    return gs_svr

### Visualization of results

The R2 is an inadequate score for nonlinear models. Thus the root mean squared error (RMSE) were employed for comparing the predictive performance between the three algorithms.

The scatter plot shows the result of the observed and predicted Papp.

In [ ]:
def performance_evaluation(model, X_test, y_test):
    #予測値
    y_pred = model.predict(X_test)
    
    print(f'Correlation coefficient between observed and predicted values: {np.corrcoef(y_pred, y_test)[0,1]}')
    
    #グラフのスケール設定
    y_min = min([min(list(y_test)), min(list(y_pred))])
    y_max = max([max(list(y_test)), max(list(y_pred))])
    
    ###実測値と予測値の散布図###
    fig = plt.figure(figsize=(4, 4))
    #散布図
    plt.scatter(y_test, y_pred)
    #タイトル
    plt.title(f'Scatter plot, RMSE: {round(np.sqrt(np.sum((y_pred - y_test)**2)/len(y_test)), 4)}', fontsize=14)
    plt.xlabel('Observed value')
    plt.ylabel('Predicted value')
    
    #グラフ間の距離の調整
    plt.grid()
    plt.show()

## Results: Fa prediction using 1D2D3D descriptors

### Confirming distributions

The distributions of this dataset in terms of molecular weight and slogp that were calculated using Mordred are shown in Figure 2 in body text. The p-values of the Wilcoxon signed rank test were higher than 0.05 in the converted Papp, MW, and slogp measurements, confirming that there was no bias in the training and test sets.

In [ ]:
X_train, X_test, y_train, y_test = distribution_conformation(df_3Ddescriptors, 'Papp')

### Model Construction

The suitable parameters were selected using GridsearchCV (cv=10, score=’neg-mean-squared-error').

In [ ]:
X_train_elastic, X_test_elastic = descriptor_preparation(X_train, X_test, y_train)

svr_model = model_construction_svr(X_train_elastic, y_train)
svr_model.best_params_

### Performance evaluation

The predicted Papp was calculated with SVR model constructed using 1D, 2D and 3D descriptor. The x-axis and y-axis show the values of converted Fa. The correlation coefficient scores between the observed and predicted values were 0.7019.

In [ ]:
performance_evaluation(svr_model, X_test_elastic, y_test)

### Contribution of descriptors for Papp prediction

Although the generated relationship between the descriptors and predicted results were effective for compound optimization, the constructed models were complex, and it was difficult to elucidate their relationships. Shapley additive explanations (SHAP) is a useful tool to overcome this hurdle, where the method calculates an important value of each feature for a prediction based on game theory. The calculation performed by the method was time- and memory-intensive due to which we did not compute the SHAP in Papp dataset. However, the script to calculate SHAP value is shown in supplemental information for Papp prediction.

In [ ]:
# load JS visualization code to notebook
shap.initjs()

# Create object that can calculate shap values
explainer = shap.KernelExplainer(svr_model.predict, pd.DataFrame(X_test_elastic))
# Calculate SHAP values
shap_values = explainer.shap_values(X_test_elastic)

shap.summary_plot(shap_values, X_test_elastic)

## References

- J-Stage: https://www.jstage.jst.go.jp/article/ciqs/2016/0/2016_Y4/_pdf/-char/ja
- github: https://github.com/mordred-descriptor/mordred
- kiseno-log: https://kiseno-log.com/2019/11/07/mordred%E3%81%A7%E8%A8%98%E8%BF%B0%E5%AD%90%E3%82%92%E8%A8%88%E7%AE%97%E3%81%97%E3%81%A6pandas%E5%BD%A2%E5%BC%8F%E3%81%A7%E5%87%BA%E5%8A%9B%E3%81%99%E3%82%8B/